# Solving simple artificial neural network to optimality in EAGO 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

In [1,2], a surrogate ANN model of bioreactor productivity was constructed by fitting results from computationally expensive CFD simulations. The author then optimized this surrogate model to obtain ideal processing conditions. This optimization problem is given by:

$
\begin{align}
    \label{prob:ANN1}
    \max_{\mathbf x\in X} B_2 + \sum_{i=1}^{3}\frac{2D_{i}}{1+\exp(-2y_{i})} \qquad \text{where} \qquad y_{i} =  B_i + \sum_{i=1}^{3}\sum_{j=1}^{8} W_{ij}x_{i}\\
\end{align}
$

We'll repeat this exercise using JuMP and EAGO.

### Input parameters

In the first block, we input parameters values supplied in the paper for $W$, $B$, $D$, and $X$ into Julia as simple array objects.

In [3]:
using JuMP, EAGO

# Box constraints for input variables
xLBD = [0.623   0.093   0.259   6.56   1114   0.013   0.127   0.004]
xUBD = [5.89    0.5     1.0     90     25000  0.149   0.889   0.049]

# Weights associated with the hidden layer
W = [ 0.54  -1.97  0.09  -2.14  1.01  -0.58  0.45  0.26;
     -0.81  -0.74  0.63  -1.60 -0.56  -1.05  1.23  0.93;
     -0.11  -0.38 -1.19   0.43  1.21   2.78 -0.06  0.40]

# Weights associated with the output layer
D = [-0.91 0.11 0.52]

# Bias associated with the hidden layer
B1 = [-2.698 0.012 2.926]

# Bias associated with the output layer
B2 = -0.46

-0.46

### Construct the JuMP model and optimize

We now formulate the problem using standard JuMP[3] syntax and optimize it. Note that we are forming an NLexpression object to handle the summation term to keep the code visually simple but this could be placed directly in the JuMP expressions instead.

In [2]:
# Model construction
model = Model(with_optimizer(EAGO.Optimizer, absolute_tolerance = 0.001))
@variable(model, xLBD[i] <= x[i=1:8] <= xUBD[i])
@NLexpression(model, prop[i=1:3], B1[i] + sum(W[i,j]*x[i] for j in 1:8))
@NLobjective(model, Max, B2 + sum(D[i]*(2/(1+exp(-2*prop[i]))) for i=1:3))

# Solves the model
optimize!(model)

Node ID: 1, Lower Bound: -Inf, Lower Variable Bounds:
             [0.623, 0.093, 0.259, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.89, 0.5, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049]
started initial preprocessing
finished initial preprocessing
finished initial lower problem

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

finished initial upper problem
finished initial postprocessing
Lower Bound (First Iteration): -0.6804021544178823, Solution: [5.89, 0.093, 1.0, 6.56, 1114.0, 0.013, 0.127, 0.004], Feasibility: true
Lower Bound (Last Iteration): -0.6804021544178823, Solution: [5.89, 0.093, 1.0, 6.56, 1114.0, 0.013, 0.127, 0.004], Feasi

Upper Bound: 0.6713409203381793, Solution: [2.55901, 0.0930043, 0.985447, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    11,        8,      -0.6733422,     -0.6734237,     -0.6713400,           8,        0.0020837,       0.0031038,        true,       true
Node ID: 14, Lower Bound: -0.6734064133195578, Lower Variable Bounds:
             [3.91488, 0.093, 0.259, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.89, 0.245625, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -0.6733173759100287, Solution: [5.89, 0.093, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Lower Bound (Last Iteration): -0.6733173759100287, Solution: [5.89, 0.093, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: Inf, Solution: [2.55901, 0.0930043, 0.985447, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: false
    12,        9,      -0.6733174,     -0.6734064,     -0.6713400,           9,        0.0020664,      

Lower Bound (Last Iteration): -0.6733422423373349, Solution: [3.91488, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: -0.641599713641462, Solution: [3.29685, 0.245614, 0.73084, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    22,       19,      -0.6733422,     -0.6733422,     -0.6713400,          19,        0.0020023,       0.0029825,        true,       true
Node ID: 18, Lower Bound: -0.6733422423384567, Lower Variable Bounds:
             [0.623, 0.093, 0.722125, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [2.68042, 0.245625, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -0.6733422423307109, Solution: [2.68042, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Lower Bound (Last Iteration): -0.6733422423307109, Solution: [2.68042, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: 0.6713404680063391, Solution: [2.18134, 0.0930043, 0.980821, 48.28, 13057.0,

Upper Bound: 0.6713397847108409, Solution: [3.29844, 0.0930088, 0.979671, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    35,       28,      -0.6716619,     -0.6733385,     -0.6713400,          28,        0.0019985,       0.0029769,        true,       true
Node ID: 54, Lower Bound: -0.6733384714258593, Lower Variable Bounds:
             [1.90889, 0.093, 0.895797, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [2.68042, 0.150234, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -0.6716618758413802, Solution: [2.68042, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Lower Bound (Last Iteration): -0.6716618758413802, Solution: [2.68042, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: -0.6595683482010843, Solution: [2.25427, 0.150227, 0.916941, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    36,       29,      -0.6716619,     -0.6733385,     -0.6713400,          29,        0.001

Upper Bound: 0.6713311976722212, Solution: [1.75585, 0.0930088, 0.876422, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    47,       28,      -0.6716583,     -0.6733348,     -0.6713400,          28,        0.0019949,       0.0029715,        true,       true
Node ID: 20, Lower Bound: -0.6733173759100287, Lower Variable Bounds:
             [3.91488, 0.093, 0.259, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.89, 0.245625, 0.439908, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -0.6731906165947669, Solution: [5.89, 0.093, 0.439908, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Lower Bound (Last Iteration): -0.6731906165947669, Solution: [5.89, 0.093, 0.439908, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: -0.6410283242510835, Solution: [4.90244, 0.245617, 0.259503, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    48,       29,      -0.6731906,     -0.6733174,     -0.6713400,          29,        0.

Lower Bound (Last Iteration): -0.6716400224904476, Solution: [5.89, 0.093, 0.722125, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: -0.6594793009982485, Solution: [4.90244, 0.150225, 0.551394, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    58,       39,      -0.6716400,     -0.6733166,     -0.6713400,          39,        0.0019766,       0.0029443,        true,       true
Node ID: 23, Lower Bound: -0.673316618074927, Lower Variable Bounds:
             [3.91488, 0.150234, 0.548453, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.89, 0.245625, 0.722125, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -Inf, Solution: [5.89, 0.093, 0.722125, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: false
Lower Bound (Last Iteration): -Inf, Solution: [5.89, 0.093, 0.722125, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: false
    59,       38,      Inf,     -0.6733166,     -0.6713400,          38,        0.0019766,       0.0029

Upper Bound: 0.6712432432371813, Solution: [3.29844, 0.0930088, 0.545641, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
Iteration   NodeID    Current_LBD     Global_LBD     Global_UBD      NodesLeft     Absolute_Gap    Absolute_Ratio     LBD_Feas     UBD_Feas
    70,       41,      -0.6715730,     -0.6732496,     -0.6713400,          41,        0.0019096,       0.0028445,        true,       true
Node ID: 81, Lower Bound: -0.6732496049722801, Lower Variable Bounds:
             [2.68042, 0.150234, 0.439908, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [3.91488, 0.245625, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -Inf, Solution: [3.91488, 0.093, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: false
Lower Bound (Last Iteration): -Inf, Solution: [3.91488, 0.093, 0.548453, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: false
    71,       40,      Inf,     -0.6732496,     -0.6713400,          40,        0.0019096,   

Upper Bound: 0.6711439078494683, Solution: [1.74672, 0.0930088, 0.438422, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: true
    83,       40,      -0.6715140,     -0.6731906,     -0.6713400,          40,        0.0018506,       0.0027566,        true,       true
Node ID: 60, Lower Bound: -0.6716618788572957, Lower Variable Bounds:
             [3.91488, 0.093, 0.895797, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.14933, 0.150234, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049]
Lower Bound (First Iteration): -0.6716618788572921, Solution: [5.14933, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Lower Bound (Last Iteration): -0.6716618788572921, Solution: [5.14933, 0.093, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049], Feasibility: true
Upper Bound: Inf, Solution: [1.74672, 0.0930088, 0.438422, 48.28, 13057.0, 0.081, 0.508, 0.0265], Feasibility: false
    84,       41,      -0.6716619,     -0.6716619,     -0.6713400,          41,        0.0003219,       0.

### Retrieve results

We then recover the objective value, the solution value, and termination status codes using standard JuMP syntax.

In [4]:
# Access calculated values
fval = JuMP.objective_value(model)
xsol = JuMP.value.(x)
status_term = JuMP.termination_status(model)
status_prim = JuMP.primal_status(model)

println("EAGO terminated with a status of $status_term and a result code of $status_prim")
println("The optimal value is: $fval, the solution found is $xsol.")

EAGO terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT
The optimal value is: 0.6713399756782958, the solution found is [2.49511, 0.0930088, 0.985666, 48.28, 13057.0, 0.081, 0.508, 0.0265].


### Reference:
1. J. D. Smith, A. A. Neto, S. Cremaschi, and D. W. Crunkleton, CFD-based optimization of a flooded bed algae bioreactor, *Industrial & Engineering Chemistry Research*, 52 (2012), pp. 7181–7188
2. A. M. Schweidtmann and A. Mitsos. Global Deterministic Optimization with Artificial Neural Networks Embedded [https://arxiv.org/pdf/1801.07114.pdf](https://arxiv.org/pdf/1801.07114.pdf)
3. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, 59 (2017), pp. 295-320.